In [3]:
# this is the new version for adding the branching strategy
import gurobipy as gp
from gurobipy import GRB
import random
import math
from itertools import combinations
import pandas as pd
# create the parameter
random.seed(1)
number_of_facility = 55 # 15
number_of_product = 45#15
time_period = 20 #15 

r_ij = {}
a_ij = {}
for i in range(0, number_of_facility ):
    for j in range(0, number_of_product):
        r_ij[i,j]=  random.randrange(12, 15) #(12, 22)
        a_ij[i,j] = random.randrange(2, 3)  # (2, 5)   # (3, 5)

d_jt = {}   
c_it = {}
capa = 600
for t in range(0, time_period  ):
    for j in range(0, number_of_product):
        d_jt[j,t] = random.randrange(350, 670)  #(30, 32)
    for i in range(0, number_of_facility  ):
        c_it[i,t] = random.randrange(500, 510)  # (500, 540)  # capa

        
s_jk = {}
q_jk = {}
for j in range(0, number_of_product):
    for k in range(0, number_of_product):
        if j == k :
            q_jk[j,k] = 9999
            s_jk[j,k] = random.randrange(7, 15)
        else:
            s_jk[j,k] = random.randrange(7, 15)
            q_jk[j,k] = random.randrange(20, 32) #(20, 32)

print('Already set up the parameter')
print(len(q_jk.keys()))
      
c_it

Already set up the parameter
2025


{(0, 0): 500,
 (1, 0): 504,
 (2, 0): 508,
 (3, 0): 507,
 (4, 0): 505,
 (5, 0): 507,
 (6, 0): 502,
 (7, 0): 507,
 (8, 0): 502,
 (9, 0): 507,
 (10, 0): 502,
 (11, 0): 503,
 (12, 0): 505,
 (13, 0): 501,
 (14, 0): 508,
 (15, 0): 509,
 (16, 0): 509,
 (17, 0): 505,
 (18, 0): 502,
 (19, 0): 506,
 (20, 0): 506,
 (21, 0): 504,
 (22, 0): 504,
 (23, 0): 503,
 (24, 0): 509,
 (25, 0): 500,
 (26, 0): 507,
 (27, 0): 505,
 (28, 0): 501,
 (29, 0): 504,
 (30, 0): 507,
 (31, 0): 506,
 (32, 0): 507,
 (33, 0): 500,
 (34, 0): 506,
 (35, 0): 504,
 (36, 0): 507,
 (37, 0): 508,
 (38, 0): 502,
 (39, 0): 505,
 (40, 0): 502,
 (41, 0): 503,
 (42, 0): 506,
 (43, 0): 501,
 (44, 0): 501,
 (45, 0): 505,
 (46, 0): 502,
 (47, 0): 507,
 (48, 0): 508,
 (49, 0): 507,
 (50, 0): 508,
 (51, 0): 502,
 (52, 0): 507,
 (53, 0): 502,
 (54, 0): 500,
 (0, 1): 506,
 (1, 1): 507,
 (2, 1): 505,
 (3, 1): 500,
 (4, 1): 507,
 (5, 1): 503,
 (6, 1): 502,
 (7, 1): 505,
 (8, 1): 502,
 (9, 1): 503,
 (10, 1): 507,
 (11, 1): 502,
 (12, 1): 504,


In [4]:
# revised version:
import random
import copy
ad_jt = copy.deepcopy(d_jt)
ac_it = copy.deepcopy(c_it)
w_ijt_ini={}
w_it_seq = {}
# print('time: ',time_period  )
for i in range(0,number_of_facility):
    for t in range(0,time_period):
        w_it_seq[i,t] = []

for t in range(0, time_period):
    for i in range(0,number_of_facility):
        for j in range(0,number_of_product):
            if ac_it[i,t] > 0 : # remain capacity > 0
                if w_it_seq[i,t]:
                    if ad_jt[j,t]/a_ij[i,j] >= ac_it[i,t]:
                        w_ijt_ini[i,j,t] = ac_it[i,t]
                        ac_it[i,t] = ac_it[i,t]-w_ijt_ini[i,j,t]
                        w_it_seq[i,t].append(j)
                        ad_jt[j,t] = ad_jt[j,t] -(w_ijt_ini[i,j,t]*a_ij[i,j])
                    else:
                        w_ijt_ini[i,j,t]=ad_jt[j,t]/a_ij[i,j]
                        ac_it[i,t] = ac_it[i,t]-w_ijt_ini[i,j,t]
                        w_it_seq[i,t].append(j)
                        ad_jt[j,t] = 0 
                else:
                    change_over_cost = 0
                    for j in range(len(w_it_seq[i,t])):
                        if j+1 < len(w_it_seq[i,t]):
                            change_over_cost = change_over_cost + q_jk[j,j+1]
                    if (ad_jt[j,t]/a_ij[i,j] ) +change_over_cost  >= ac_it[i,t]:
                        w_ijt_ini[i,j,t] = c_it[i,t]-change_over_cost
                        w_it_seq[i,t].append(j)
                        ad_jt[j,t] = ad_jt[j,t] -(w_ijt_ini[i,j,t]*a_ij[i,j])
                        ac_it[i,t] = ac_it[i,t]-w_ijt_ini[i,j,t]
                    else:
                        w_ijt_ini[i,j,t]=ad_jt[j,t]/a_ij[i,j]
                        ac_it[i,t] = ac_it[i,t]-w_ijt_ini[i,j,t]
                        w_it_seq[i,t].append(j)
                        ad_jt[j,t] = 0 
            else:
                break # go to next factory
                
for i in range(0,number_of_facility):
    for j in range(0,number_of_product):
        for t in range(0, time_period):
            try:
                if w_ijt_ini[i,j,t]:
                    pass
            except KeyError:
                w_ijt_ini[i,j,t] = 0
chk =0
for  j in  range(0,number_of_product):
    for t in range(0, time_period):
        if ad_jt[j,t] >0:
            chk = chk +1 
        else:
            pass
if chk == 0:
    print('Fufill demand, find the feasible solution! ')
else:
    print('Infeasible solution! ')
_su = 0
for i in w_ijt_ini:
    _su += w_ijt_ini[i]
_su

Fufill demand, find the feasible solution! 


231457.5

In [ ]:
m = gp.Model('Original Problem')
# create decision var.
w_ijt = m.addVars(number_of_facility, number_of_product,time_period,vtype=GRB.CONTINUOUS, name = "w"  )
x_ijt= m.addVars(number_of_facility, number_of_product,time_period,vtype=GRB.BINARY, name = "x"  )
y_ijkt = m.addVars(number_of_facility, number_of_product,number_of_product,time_period,vtype=GRB.BINARY, name = "y" )
z_ijt =  m.addVars(number_of_facility, number_of_product,time_period,vtype=GRB.BINARY, name = "z"  )
v_ijt =  m.addVars(number_of_facility, number_of_product,time_period,vtype=GRB.INTEGER,lb=1, name = "v"  )

# create the obj. function:
m.setObjective(gp.quicksum(r_ij[i,j]* w_ijt[i,j,t] for i in range(0, number_of_facility ) for j in range(0, number_of_product) for t in range(0, time_period)),GRB.MINIMIZE)
# create the constraint:
# constraint 1:
for j in range(0, number_of_product):
    for t in range(0, time_period):
        m.addConstr(gp.quicksum(a_ij[i,j] * w_ijt[i,j,t] for i in range(0, number_of_facility )) == d_jt[j,t],"demand%s,%s"%(j,t))

        
# constrain 2:
for i in range(0, number_of_facility):
    for t in range(0, time_period):
        m.addConstr(gp.quicksum(w_ijt[i,j,t] for j in  range(0, number_of_product)) + 
                    gp.quicksum(q_jk[j,k]*y_ijkt[i,j,k,t] for k in range(0, number_of_product)  for j in  range(0, number_of_product) if j != k ) <= c_it[i,t],"demand_fulfill %s,%s"%(i,t) )
# constaints 3:
for i in range(0, number_of_facility):
    for j  in range(0, number_of_product):
        for t in range(0, time_period):
            m.addConstr(x_ijt[i,j,t] <= z_ijt[i,j,t]+ gp.quicksum(y_ijkt[i,j,k,t]  for k in  range(0, number_of_product) if k != j),"constrint3 %s,%s,%s"%(i,j,t))
# constraint 4:
for i in range(0, number_of_facility):
    for j  in range(0, number_of_product):
        for t in range(0, time_period):
            m.addConstr(a_ij[i,j]*w_ijt[i,j,t] <= d_jt[j,t] * x_ijt[i,j,t],"constraint4 %s,%s,%s"%(i,j,t))
# constraint 5:
for i in range(0, number_of_facility):
    for t in range(0, time_period):
        m.addConstr(gp.quicksum(z_ijt[i,j,t] for j in range(0, number_of_product) ) == 1 ,"constraint5 %s,%s"%(i,t))
#contstraint 6:
for i in range(0, number_of_facility):
    for j in range(0, number_of_product):
        for t in range(0, time_period-1):
            m.addConstr(z_ijt[i,j,t] + gp.quicksum(y_ijkt[i,k,j,t] for k in range(0, number_of_product) if k!= j ) == z_ijt[i,j,t+1]+ gp.quicksum(y_ijkt[i,j,k,t] for k in range(0, number_of_product) if k!= j ),"constraint6 %s,%s,%s"%(i,j,t))
            
# constraint 7:
for i in range(0, number_of_facility):
    for j in range(0, number_of_product):
        for k in range(0, number_of_product):
            for t in range(0, time_period):
                # m.addConstr(v_ijt[i,k,t] >= v_ijt[i,j,t] +1 - number_of_product*(1- y_ijkt[i,j,k,t]),"constraint7 %s,%s,%s,%s"%(i,j,k,t))
                m.addConstr(v_ijt[i,j,t] +1 - number_of_product+ number_of_product * y_ijkt[i,j,k,t] <= v_ijt[i,k,t],"constraint7 %s,%s,%s,%s"%(i,j,k,t))




m.update()
print('Successful Build model!')
# m.display()  # do not display, it requrires lots of memory
m.Params.timeLimit = 1800.0
m.optimize()
m.objval
for i in m.getVars():
    print(i.X,i.VarName)

# print(m.getAttr("X",m.getVars()))
# print(m.getAttr("VarName",m.getVars()))
# w_ijt = m.addVars((number_of_facility,number_of_product,time_period),GRB.CONTINUOUS)

In [6]:
a = {1:2,2:3,5:8}
len(a.keys())

dic = {1:3,2:5,3:4}
def test(dic):
    dic[len(dic.keys())+1] = 8
    return dic
test(dic)
dic 


# wijt is the input for the masterproblem:
# wijt * rij and sum up all the j and t --->這是一條column
# (remark: sum j,t for wijt * ij would be the coeffecient of the lamda)
# cost_for_lamda = {0:25,1:33,2:32}---->
# 但lamda 維度應該沒有那麼簡單, 維度應該是lambda(i,column XX) (i.e. plant i的第XX 解)
# 存一個參數dictinary or DF?
# lamda_ik ={[0:0]:23,[2,1]:32} 
lamda_ik ={(0,0):23,(2,1):32} 
print(lamda_ik.keys())
class MasterProblem:
    def __init__(self,column,c_in,a_w_ijtn,a_ij,r_ij,number_of_product,time_period):
        self.model = gp.Model("MasterProblem")
        self.number_of_product = number_of_product
        self.a_w_ijtn = a_w_ijtn # used for demand constraint
        self.column = column
        self.subProblemVauleSum = {} # 儲存每一個Subproblem 每一次的決策變數加總,會用來加進去 產生限制式,避免產生相同的column
        self.a_ij =a_ij
        self.r_ij =r_ij
        self.time_period = time_period
        self.number_of_facility =  number_of_facility
        self.c_in = c_in #cost_for_lamda 該是甚麼型態? 感覺應該存成一個dictionaty?  類似原本參數的概念,拿來與lamda相乘
    def buildModel(self):
        self.transfer_Column_into_list()
        self.generateVariables()
        self.generateConstraints()
        self.generateObjective()
        self.model.update()
    def generateObjective(self):
        self.model.setObjective(gp.quicksum(self.columnVar[i,n]* self.c_in[i,n] for i in range(0, number_of_facility) for n in range(0,len(self.column[i]))),GRB.MINIMIZE)
        
    def transfer_Column_into_list(self):
        return ([(col,self.column[col][inner])  for col in self.column for inner in self.column[col]])
    def generateVariables(self): # 0521 change var type from continuous to integer , 搭配 solveRelaxedModel 函數改變
        self.columnVar = self.model.addVars(self.transfer_Column_into_list(),lb = 0,vtype = GRB.BINARY, name = "lamda")
    def generateConstraints(self):
        for j in range(0, self.number_of_product):
            for t in range(0, time_period):
                self.model.addConstr(gp.quicksum(self.a_w_ijtn[i,j,t,n]*self.columnVar[i,n] for i in range(0, number_of_facility ) for n in range(0,len(self.column[i]))) >= d_jt[j,t],"master_constraint1 %s,%s"%(j,t))
        for i in range(0, self.number_of_facility ):
            self.model.addConstr(gp.quicksum(self.columnVar[i,n] *(-1) for n in range(0,len(self.column[i]) ) )  >= -1,"master_constraint2 %s"%(i)  )
    def solveRelaxedModel(self):
        self.model.update()
        self.relaxModel = self.model.relax() # 521  change
        self.relaxModel.optimize() # 521 
    def solveModel(self):
        self.model.optimize()
    def getDuals(self):
        return self.relaxModel.getAttr("Pi", self.relaxModel.getConstrs())
    def addColumn(self, wijt,facility):
        self.column[facility].append(len(self.column[facility]))
        sum_j_t =0
        column_aw= []
        for j in range(0, self.number_of_product):
            for t in range(0, time_period):
                column_aw.append(wijt[facility,j,t] * self.a_ij[facility,j])
                self.a_w_ijtn[facility,j,t,len(self.column[facility]) -1] = wijt[facility,j,t] * self.a_ij[facility,j]
                sum_j_t = sum_j_t +   wijt[facility,j,t] * self.r_ij[facility,j]
        self.c_in[facility,len(self.column[facility]) -1] = sum_j_t
        # 改變self.column,self.c_in,self.a_w_ijtn
        c = gp.Column()
        c.addTerms(column_aw+[-1 if i==facility else 0 for i in range(0,self.number_of_facility) ],self.model.getConstrs())
        self.model.addVar(vtype = GRB.BINARY, obj = self.c_in[facility,len(self.column[facility]) -1],column = c,name ="lamba[%s,%s]"%(facility,len(self.column[facility]) -1)  )
        self.model.update()
    def addColumn2(self):
        c = gp.Column()
        c.addTerms(column_aw+[1 if i==facility else 0 for i in range(0,self.number_of_facility) ],self.model.getConstrs())
        self.model.addVar(vtype = GRB.CONTINUOUS, obj = self.c_in[facility,len(self.column[facility]) -1],column = c,name ="lamba[%s,%s]"%(facility,len(self.column[facility]) -1)  )
        self.model.update()
    def test1(self, wijt,facility):
#         self.column[facility].append(len(self.column[facility]))
        self.sum_j_t =0
#         self.column_aw= []
        for j in range(0, self.number_of_product):
            for t in range(0, time_period):
#                 self.column_aw.append(wijt[facility,j,t] * self.a_ij[facility,j])
#                 self.a_w_ijtn[facility,j,t,len(self.column[facility]) -1] = wijt[facility,j,t] * self.a_ij[facility,j]
                self.sum_j_t = self.sum_j_t +   wijt[facility,j,t] * self.r_ij[facility,j]
        self.addcol = True
        if len(self.column[facility]) >= 10:
            _same_column = 0
            for n in range(0,len(self.column[facility]) ):
                if self.sum_j_t == self.c_in[facility,n]:
                    _same_column += 1 
            if _same_column >= 1000:  # (len(self.column[facility])) //2
                self.addcol= False
#             else:
#                 self.c_in[facility,len(self.column[facility]) -1] = sum_j_t
#         else:
#             self.c_in[facility,len(self.column[facility]) -1] = sum_j_t
        # 改變self.column,self.c_in,self.a_w_ijtn
#         c = gp.Column()
#         c.addTerms(self.column_aw+[1 if i==facility else 0 for i in range(0,self.number_of_facility) ],self.model.getConstrs())
#         self.model.addVar(vtype = GRB.CONTINUOUS, obj = self.c_in[facility,len(self.column[facility]) -1],column = c,name ="lamba[%s,%s]"%(facility,len(self.column[facility]) -1)  )
#         self.model.update()
        
    def test2(self,facility,wijt):
        if self.addcol == True:
            self.skip =False
            self.column[facility].append(len(self.column[facility]))
            self.column_aw= []
            for j in range(0, self.number_of_product):
                for t in range(0, time_period):
                    self.column_aw.append(wijt[facility,j,t] * self.a_ij[facility,j])
                    self.a_w_ijtn[facility,j,t,len(self.column[facility]) -1] = wijt[facility,j,t] * self.a_ij[facility,j]
            self.c_in[facility,len(self.column[facility]) -1] = self.sum_j_t
            c = gp.Column()
            c.addTerms(self.column_aw+[-1 if i==facility else 0 for i in range(0,self.number_of_facility) ],self.model.getConstrs())
            self.model.addVar(vtype = GRB.CONTINUOUS, obj = self.c_in[facility,len(self.column[facility]) -1],column = c,name ="lamba[%s,%s]"%(facility,len(self.column[facility]) -1)  )
            self.model.update()
        else: 
            self.skip = True 

        
                    
        

class SubProblem:
    def __init__(self, duals,number_of_facility,facility,number_of_product,time_period,r_ij,d_jt,c_it,a_ij,s_jk,
                 q_jk):
        self.duals = duals
        self.facility = facility
        self.number_of_facility = number_of_facility
        self.number_of_product = number_of_product
        self.time_period =time_period
        self.r_ij = r_ij
        self.d_jt =d_jt
        self.a_ij = a_ij
        self.s_jk=s_jk
        self.q_jk = q_jk
        self.c_it = c_it
        self.model = gp.Model("SubProblem")        
    def buildModel(self):
        self.generateVariables()
        self.generateConstraints()
        self.generateObjective()
        self.model.update()
    def generateVariables(self):
        self.SubProblem_wijt = self.model.addVars([self.facility],self.number_of_product,self.time_period,lb=0,vtype = GRB.CONTINUOUS,name = "pricing_wijt")
        self.SubProblem_xijt = self.model.addVars([self.facility],self.number_of_product,self.time_period,vtype = GRB.BINARY, name = "pricing_xijt")
        self.SubProblem_yijkt = self.model.addVars([self.facility],self.number_of_product,self.number_of_product,self.time_period,vtype = GRB.BINARY,name = "pricing_yijkt")
        self.SubProblem_zijt = self.model.addVars([self.facility],self.number_of_product,self.time_period,lb=0,vtype = GRB.BINARY,name = "pricing_zijt")
        self.SubProblem_vijt = self.model.addVars([self.facility],self.number_of_product,self.time_period,lb=1,vtype = GRB.INTEGER,name = "pricing_vijt")
    def generateConstraints(self):
        # refer by page 20 PPT:
        # constraints 20:
        for t in  range(0, self.time_period):
            self.model.addConstr(gp.quicksum(self.SubProblem_wijt[self.facility,j,t] for j in range(0, self.number_of_product) )  + gp.quicksum(self.q_jk[j,k]*self.SubProblem_yijkt[self.facility,j,k,t] for j in range(0, self.number_of_product) for k in range(0, self.number_of_product))<= self.c_it[self.facility,t],"princing_constraint1 %s,%s"%(self.facility,t))       
        # constaints 21:
        for j  in range(0, self.number_of_product):
            for t in range(0, self.time_period):
                self.model.addConstr(self.SubProblem_xijt[self.facility,j,t] <= self.SubProblem_zijt[self.facility,j,t]+ gp.quicksum(self.SubProblem_yijkt[self.facility,j,k,t]  for k in  range(0, self.number_of_product) if k != j),"pricing_constrint 21 %s,%s,%s"%(self.facility,j,t))
        # constraint 22: # 小於改大於
        for j  in range(0, self.number_of_product):
            for t in range(0, self.time_period):
                self.model.addConstr(self.a_ij[self.facility,j]*self.SubProblem_wijt[self.facility,j,t] <= self.d_jt[j,t] * self.SubProblem_xijt[self.facility,j,t],"constraint22 %s,%s,%s"%(self.facility,j,t))
        # constraint 23:
        for t in range(0, self.time_period):
            self.model.addConstr(gp.quicksum(self.SubProblem_zijt[self.facility,j,t] for j in range(0, self.number_of_product) ) == 1 ,"constraint23 %s,%s"%(self.facility,t))
        #contstraint 24:
        for j in range(0, self.number_of_product):
            for t in range(0, self.time_period-1):
                self.model.addConstr(self.SubProblem_zijt[self.facility,j,t] + gp.quicksum(self.SubProblem_yijkt[self.facility,k,j,t] for k in range(0, self.number_of_product) if k!= j ) == self.SubProblem_zijt[self.facility,j,t+1]+ gp.quicksum(self.SubProblem_yijkt[self.facility,j,k,t] for k in range(0, self.number_of_product) if k!= j ),"pricing constraint24 %s,%s,%s"%(self.facility,j,t))
        # constraint 25:
        for j in range(0, self.number_of_product):
            for k in range(0, self.number_of_product):
                for t in range(0, self.time_period):
                    # m.addConstr(v_ijt[i,k,t] >= v_ijt[i,j,t] +1 - number_of_product*(1- y_ijkt[i,j,k,t]),"constraint7 %s,%s,%s,%s"%(i,j,k,t))
                    self.model.addConstr(self.SubProblem_vijt[self.facility,j,t] +1 - self.number_of_product+ self.number_of_product * self.SubProblem_yijkt[self.facility,j,k,t] <= self.SubProblem_vijt[self.facility,k,t],"pricing constraint25 %s,%s,%s,%s"%(self.facility,j,k,t))
    def generateObjective(self):
        # partition the dual from the two part
        self.duals_jt= {}
        self.duals_i = {}
        _cnt = 0 
        for j in range(0, self.number_of_product):
            for t in range(0, self.time_period):
                self.duals_jt[j,t] = self.duals[_cnt]
                _cnt +=1
        for i in range(0,self.number_of_facility):
            self.duals_i[i] = self.duals[_cnt]
            _cnt +=1
            
        self.model.setObjective(gp.quicksum(self.r_ij[self.facility,j] *self.SubProblem_wijt[self.facility,j,t] 
                                            for j in range(0, self.number_of_product)  for t in range(0, self.time_period) ) -
                                gp.quicksum(self.duals_jt[j,t]*self.a_ij[self.facility,j]*
                                            self.SubProblem_wijt[self.facility,j,t] for j in range(0, self.number_of_product) 
                                            for t in range(0, self.time_period) ) + self.duals_i[self.facility],GRB.MINIMIZE)
    def solveModel(self,timeLimit ,EPSILON):
        self.model.setParam('TimeLimit', timeLimit)
        self.model.setParam('MIPGAP', EPSILON)
        self.model.optimize()
    def obtain_column(self):
        wijt_solution = {}
        cnt = 0 
        for j in range(0,self.number_of_product):
            for t in range(0,self.time_period):
                wijt_solution[self.facility,j,t] = self.model.getVars()[cnt].X
                cnt = cnt +1 
        return wijt_solution
    def obtain_pricing_solution(self):
        return self.model.objval, self.model.getAttr("X",self.model.getVars()),self.model.getAttr("VarName",self.model.getVars())
    
        
class CreateInitialSolution:
    def __init__(self, ini_wijt,number_of_facility,number_of_product,time_period,a_ij,r_ij):
        self.a_ij =a_ij
        self.r_ij =r_ij
        self.ini_wijt = ini_wijt
        self.number_of_facility = number_of_facility
        self.number_of_product = number_of_product
        self.time_period = time_period
    def crateInitialColumn(self):
        column = {}
        c_in = {}
        a_w_ijtn= {}
        for i in range(0,self.number_of_facility):
            column[i] = []
            column[i].append(0)
            c_in[i,0] = 0
            sum_j_t = 0
            for j in range(0,self.number_of_product):
                for t in range(0,self.time_period):
                    a_w_ijtn[i,j,t,0] = self.ini_wijt[i,j,t]* self.a_ij[i,j]
                    sum_j_t = sum_j_t + (self.ini_wijt[i,j,t]* self.r_ij[i,j])
            c_in[i,0] = sum_j_t # store the coefficient
        return column,c_in,a_w_ijtn

# branching:
def branching():
    _var = {}
    for v in masterProblem.relaxModel.getVars():
        if v.x ==1 :
            _var[v.VarName] = 0 
        elif v.x ==0 :
            _var[v.VarName] = 0 
        elif v.x < 0.5 :
            _var[v.VarName] = 0.5 -v.x 
        else :
            _var[v.VarName] = v.x - 0.5 
    most_fractional = max(_var, key = _var.get) # obtain the variable name
    return most_fractional
    
    
            
        
        
    

dict_keys([(0, 0), (2, 1)])


In [10]:
initial = CreateInitialSolution(w_ijt_ini,number_of_facility,number_of_product,time_period,a_ij,r_ij)
initial_column,c_in,a_w_ijtn =initial.crateInitialColumn()
masterProblem = MasterProblem(initial_column,c_in,a_w_ijtn,a_ij,r_ij,number_of_product,time_period)
masterProblem.buildModel()
masterProblem.solveRelaxedModel()
count = 0 

direction = True
cnt_column = 0
obj_val = []
while direction == True and cnt_column <= 400:
    direcition = False
    if count == 0 :
        print('-----------')
        masterdual = masterProblem.getDuals()
        pass
    else:
            
        masterProblem.solveRelaxedModel()
        print('------OBJ:----------', masterProblem.relaxModel.objval)
        obj_val.append(masterProblem.relaxModel.objval)
        masterdual = masterProblem.getDuals()
    count =1 

    for i in range(0,initial.number_of_facility):
        subProblem = SubProblem(masterdual,number_of_facility,i,number_of_product,time_period,r_ij,d_jt,c_it,a_ij,s_jk,
                     q_jk)
        subProblem.buildModel()
        subProblem.solveModel(130,0.10)
        if subProblem.obtain_pricing_solution()[0] < 0:
            cnt_column += 1 
            direction = True
            subProblem.obtain_pricing_solution()
            price_solution =subProblem.obtain_column()
            try:
                masterProblem.subProblemVauleSum[i].append(sum([i.x for i in subProblem.model.getVars() if i.VarName[0:12] == 'pricing_wijt' ]))
                # masterProblem.subProblemVauleSum[i].append([i.x for i in subProblem.model.getVars()])
            except KeyError:
                masterProblem.subProblemVauleSum[i] = []
                masterProblem.subProblemVauleSum[i].append(sum([i.x for i in subProblem.model.getVars()  if i.VarName[0:12] == 'pricing_wijt']))
                # masterProblem.subProblemVauleSum[i].append([i.x for i in subProblem.model.getVars()])
                
            
#             masterProblem.test1(price_solution,i)
#             masterProblem.test2(i,price_solution)
#             # masterProblem.model.display()
#             if masterProblem.skip == True:
#                 continue
#             else:
#                 pass
            #----
            masterProblem.addColumn(price_solution,i)
            # masterProblem.model.display()
            #----
        else:
            pass
# masterProblem.solveRelaxedModel()
masterProblem.model.Params.timeLimit = 600

# masterProblem.solveModel()
masterProblem.solveRelaxedModel()
masterProblem.model.objval 

Gurobi Optimizer version 9.5.1 build v9.5.1rc2 (win64)
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads
Optimize a model with 955 rows, 55 columns and 1403 nonzeros
Model fingerprint: 0x70abe8cf
Coefficient statistics:
  Matrix range     [1e+00, 7e+02]
  Objective range  [1e+03, 1e+05]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 7e+02]
Presolve removed 955 rows and 55 columns
Presolve time: 0.01s
Presolve: All rows and columns removed
Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    2.9977090e+06   0.000000e+00   0.000000e+00      0s

Solved in 0 iterations and 0.01 seconds (0.00 work units)
Optimal objective  2.997709000e+06
-----------
Set parameter TimeLimit to value 130
Set parameter MIPGap to value 0.1
Gurobi Optimizer version 9.5.1 build v9.5.1rc2 (win64)
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads
Optimize a model with 43195 rows, 44100 columns and 282150 nonzeros
Model fingerp

Presolved: 38564 rows, 39425 columns, 229555 nonzeros
Variable types: 23 continuous, 39402 integer (38545 binary)

Deterministic concurrent LP optimizer: primal and dual simplex
Showing first log only...

Extra simplex iterations after uncrush: 10
Concurrent spin time: 0.00s

Solved with primal simplex

Root relaxation: objective -1.353953e+06, 1242 iterations, 0.37 seconds (0.31 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0 -1353952.7    0   49    0.00000 -1353952.7      -     -    1s
H    0     0                    -649134.5000 -1353952.7   109%     -    1s
H    0     0                    -758119.5577 -1353952.7  78.6%     -    2s
H    0     0                    -903055.0000 -1353952.7  49.9%     -    2s
     0     0 -1326957.4    0  125 -903055.00 -1326957.4  46.9%     -    3s
H    0     0                    -926707.2184 -1326957.4  43.2%     -    3s
H  

Set parameter MIPGap to value 0.1
Gurobi Optimizer version 9.5.1 build v9.5.1rc2 (win64)
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads
Optimize a model with 43195 rows, 44100 columns and 282150 nonzeros
Model fingerprint: 0x73d328de
Variable types: 900 continuous, 43200 integer (42300 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+04]
  Objective range  [5e+00, 7e+02]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 5e+02]
Found heuristic solution: objective 0.0000000
Presolve removed 4631 rows and 4675 columns
Presolve time: 0.36s
Presolved: 38564 rows, 39425 columns, 229555 nonzeros
Variable types: 23 continuous, 39402 integer (38545 binary)

Deterministic concurrent LP optimizer: primal and dual simplex
Showing first log only...

Extra simplex iterations after uncrush: 10
Concurrent spin time: 0.01s

Solved with primal simplex

Root relaxation: objective -1.353854e+06, 1188 iterations, 0.50 seconds (0.32 work units)

    Nodes


Cutting planes:
  Gomory: 7
  MIR: 32
  Flow cover: 2
  Zero half: 4
  RLT: 1

Explored 1 nodes (4046 simplex iterations) in 4.47 seconds (1.93 work units)
Thread count was 8 (of 8 available processors)

Solution count 6: -1.24732e+06 -1.15439e+06 -1.12102e+06 ... 0

Optimal solution found (tolerance 1.00e-01)
Best objective -1.247318396862e+06, best bound -1.321321339111e+06, gap 5.9330%
Set parameter TimeLimit to value 130
Set parameter MIPGap to value 0.1
Gurobi Optimizer version 9.5.1 build v9.5.1rc2 (win64)
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads
Optimize a model with 43195 rows, 44100 columns and 282150 nonzeros
Model fingerprint: 0xdff0472f
Variable types: 900 continuous, 43200 integer (42300 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+04]
  Objective range  [5e+00, 7e+02]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 5e+02]
Found heuristic solution: objective 0.0000000
Presolve removed 4631 rows and 4675 colu

     0     0 -1316260.9    0  156 -1198025.6 -1316260.9  9.87%     -    6s

Cutting planes:
  Learned: 1
  Gomory: 11
  Lift-and-project: 3
  MIR: 72
  Flow cover: 5
  Zero half: 5
  Mod-K: 3
  RLT: 2

Explored 1 nodes (5043 simplex iterations) in 6.62 seconds (2.39 work units)
Thread count was 8 (of 8 available processors)

Solution count 5: -1.19803e+06 -1.1948e+06 -917492 ... 0

Optimal solution found (tolerance 1.00e-01)
Best objective -1.198025572205e+06, best bound -1.316260865726e+06, gap 9.8692%
Set parameter TimeLimit to value 130
Set parameter MIPGap to value 0.1
Gurobi Optimizer version 9.5.1 build v9.5.1rc2 (win64)
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads
Optimize a model with 43195 rows, 44100 columns and 282150 nonzeros
Model fingerprint: 0x78288829
Variable types: 900 continuous, 43200 integer (42300 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+04]
  Objective range  [4e+00, 7e+02]
  Bounds range     [1e+00, 1e+00]
  RH

...

Concurrent spin time: 0.06s

Solved with primal simplex

Root relaxation: objective -2.127674e+06, 2046 iterations, 0.49 seconds (0.39 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0 -2127673.9    0   78    0.00000 -2127673.9      -     -    1s
H    0     0                    -1157281.500 -2127673.9  83.9%     -    1s
     0     0 -2084982.9    0  152 -1157281.5 -2084982.9  80.2%     -    3s
H    0     0                    -1545567.500 -2084982.9  34.9%     -    5s
     0     0 -2084351.2    0  146 -1545567.5 -2084351.2  34.9%     -    5s
H    0     0                    -1693473.158 -2084341.4  23.1%     -    5s
     0     0 -2084341.4    0  148 -1693473.2 -2084341.4  23.1%     -    5s
     0     0 -2084301.3    0  151 -1693473.2 -2084301.3  23.1%     -    5s
H    0     0                    -1723692.538 -2077591.3  20.5%     -    6s
     0     0 -2075054.8   

H    0     0                    -1158046.000 -2080898.8  79.7%     -    5s
H    0     0                    -1648591.220 -2080898.8  26.2%     -    5s
     0     0 -2080636.5    0  160 -1648591.2 -2080636.5  26.2%     -    5s
     0     0 -2080636.5    0  160 -1648591.2 -2080636.5  26.2%     -    5s
     0     0 -2071828.1    0  224 -1648591.2 -2071828.1  25.7%     -    6s
     0     0 -2070993.6    0  224 -1648591.2 -2070993.6  25.6%     -    7s
     0     0 -2070993.6    0  226 -1648591.2 -2070993.6  25.6%     -    7s
H    0     0                    -1661847.556 -2070993.6  24.6%     -    8s
H    0     0                    -1737549.405 -2068254.9  19.0%     -    8s
     0     0 -2068254.9    0  232 -1737549.4 -2068254.9  19.0%     -    8s
H    0     0                    -1969047.751 -2068254.9  5.04%     -   10s

Cutting planes:
  Learned: 1
  Gomory: 7
  Lift-and-project: 1
  MIR: 161
  Zero half: 11
  Mod-K: 1
  RLT: 17
  Relax-and-lift: 1

Explored 1 nodes (7699 simplex iterations)

H    0     0                    -2032202.916 -2068322.8  1.78%     -    8s
     0     0 -2068322.8    0  186 -2032202.9 -2068322.8  1.78%     -    8s

Cutting planes:
  Learned: 3
  Gomory: 10
  MIR: 76
  Flow cover: 7
  Zero half: 12
  RLT: 4
  Relax-and-lift: 3

Explored 1 nodes (7131 simplex iterations) in 8.10 seconds (3.05 work units)
Thread count was 8 (of 8 available processors)

Solution count 7: -2.0322e+06 -1.54873e+06 -1.54675e+06 ... 0

Optimal solution found (tolerance 1.00e-01)
Best objective -2.032202915921e+06, best bound -2.068322814259e+06, gap 1.7774%
Set parameter TimeLimit to value 130
Set parameter MIPGap to value 0.1
Gurobi Optimizer version 9.5.1 build v9.5.1rc2 (win64)
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads
Optimize a model with 43195 rows, 44100 columns and 282150 nonzeros
Model fingerprint: 0x365e69af
Variable types: 900 continuous, 43200 integer (42300 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+04]
  Ob

H    0     0                    -1277575.232 -2124964.5  66.3%     -    2s
H    0     0                    -1432821.882 -2124964.5  48.3%     -    3s
H    0     0                    -1744139.431 -2124964.5  21.8%     -    3s
     0     0 -2081701.9    0  139 -1744139.4 -2081701.9  19.4%     -    4s
H    0     0                    -1950162.904 -2081701.9  6.75%     -    5s

Cutting planes:
  Learned: 1
  Gomory: 5
  MIR: 40
  Flow cover: 2
  Zero half: 8
  Relax-and-lift: 1

Explored 1 nodes (6740 simplex iterations) in 5.80 seconds (2.18 work units)
Thread count was 8 (of 8 available processors)

Solution count 5: -1.95016e+06 -1.74414e+06 -1.43282e+06 ... 0

Optimal solution found (tolerance 1.00e-01)
Best objective -1.950162903696e+06, best bound -2.081701904447e+06, gap 6.7450%
Set parameter TimeLimit to value 600
Gurobi Optimizer version 9.5.1 build v9.5.1rc2 (win64)
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads
Optimize a model with 955 rows, 495 colu

AttributeError: Unable to retrieve attribute 'objval'

In [12]:
branching()
# masterProblem.relaxModel.getVars()

'lamda[0,0]'